In [2]:
# Burgers Equation 
import torch
import torch.nn as nn
import numpy as np
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.hidden = nn.Sequential(
            nn.Linear(2, 20),
            nn.Tanh(),
            nn.Linear(20, 20),
            nn.Tanh(),
            nn.Linear(20, 20),
            nn.Tanh(),
            nn.Linear(20, 1)
            
        )
    def forward(self, x, t):
        inputs = torch.cat([x, t], dim = 1)
        u = self.hidden(inputs)
        return u

In [3]:
# define the pde residual function
def pde_residual(x, t, model, nu = 0.01):
    #enable gradients for x and t so we can compute their derivatives
    x.requires_grad = True
    t.requires_grad = True
    
    u = model(x, t) #predict u from the model
    u_t = torch.autograd.grad(u, t, torch.ones_like(u), create_graph=True)[0]  
    u_x = torch.autograd.grad(u, x, torch.ones_like(u), create_graph=True)[0]  
    u_xx = torch.autograd.grad(u_x, x, torch.ones_like(u_x), create_graph=True)[0]  

    
    residual = u_t + u * u_x - nu* u_xx  #residual of burger's equation
    return residual

#initial and boundary condiitons
def initial_condition(x):
    return -torch.sin(np.pi * x)

def boundary_condition(x, t):
    return torch.zeros_like(t)

In [4]:
#create the training data
x = torch.linspace(-1, 1, 200).view(-1, 1)  #spatial points
t = torch.linspace(0, 1, 100).view(-1, 1)  #temporal points
x_train, t_train = torch.meshgrid(x.squeeze(), t.squeeze(), indexing = 'xy')
x_train = x_train.reshape(-1, 1)
t_train = t_train.reshape(-1, 1)

In [5]:
import torch.optim as optim
import torch.nn as nn
loss_fn = nn.MSELoss()


model = PINN()
optimizer = optim.Adam(model.parameters(), lr = 0.01)

In [6]:
x = torch.linspace(-1, 1, 200).view(-1, 1)  #spatial points
t = torch.linspace(0, 1, 100).view(-1, 1)  #temporal points
x_train, t_train = torch.meshgrid(x.squeeze(), t.squeeze(), indexing = 'xy')
x_train = x_train.reshape(-1, 1)
t_train = t_train.reshape(-1, 1)

In [7]:
num_epochs = 12000
for each in range(num_epochs): 
    model.train()
    
    # Compute initial condition loss
    u_pred = model(x, torch.zeros_like(x)) 
    u_true = initial_condition(x)
    loss_ic = torch.mean((u_pred - u_true)**2)
    
    # Compute boundary condition loss
    u_pred_left = model(torch.full_like(t, -1), t) # u(-1, t)
    u_pred_right = model(torch.full_like(t, 1), t) # u(1, t)
    loss_bc = torch.mean((u_pred_left - boundary_condition(torch.full_like(t, -1), t))**2) + \
              torch.mean((u_pred_right - boundary_condition(torch.full_like(t, 1), t))**2)
    
    # Compute PDE residual loss
    residual = pde_residual(x_train, t_train, model)
    loss_pde = torch.mean(residual**2) 
    
    loss = loss_ic + loss_bc + loss_pde
    
    # Backpropagation and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
#     print("Hi")
   
    if each % 500 == 0:
        print(f'Epoch [{each + 1}/{num_epochs}], Loss: {loss.item():.4f}')

      


Epoch [1/12000], Loss: 0.9652
Epoch [501/12000], Loss: 0.0109
Epoch [1001/12000], Loss: 0.0025
Epoch [1501/12000], Loss: 0.0013
Epoch [2001/12000], Loss: 0.0008
Epoch [2501/12000], Loss: 0.0010
Epoch [3001/12000], Loss: 0.0024
Epoch [3501/12000], Loss: 0.0006
Epoch [4001/12000], Loss: 0.0017
Epoch [4501/12000], Loss: 0.0030
Epoch [5001/12000], Loss: 0.0023
Epoch [5501/12000], Loss: 0.0003
Epoch [6001/12000], Loss: 0.0017
Epoch [6501/12000], Loss: 0.0001
Epoch [7001/12000], Loss: 0.0002
Epoch [7501/12000], Loss: 0.0002
Epoch [8001/12000], Loss: 0.0001
Epoch [8501/12000], Loss: 0.0001
Epoch [9001/12000], Loss: 0.0001
Epoch [9501/12000], Loss: 0.0001
Epoch [10001/12000], Loss: 0.0002
Epoch [10501/12000], Loss: 0.0007
Epoch [11001/12000], Loss: 0.0008
Epoch [11501/12000], Loss: 0.0002
